<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Simple-example:-monkeypatching-functions" data-toc-modified-id="Simple-example:-monkeypatching-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Simple example: monkeypatching functions</a></span></li><li><span><a href="#Example:-preventing--&quot;requests&quot;-from-remote-operations" data-toc-modified-id="Example:-preventing--&quot;requests&quot;-from-remote-operations-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example: preventing  "requests" from remote operations</a></span></li><li><span><a href="#Things-that-might-break-pytest" data-toc-modified-id="Things-that-might-break-pytest-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Things that might break pytest</a></span></li><li><span><a href="#Additional-references" data-toc-modified-id="Additional-references-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Additional references</a></span></li></ul></div>

# Monkey patching/mocking modules and environments

[Monkey patching/mocking modules and environments](https://docs.pytest.org/en/latest/monkeypatch.html)

Sometimes tests need to invoke functionality that depends on global settings of which invokes code that cannot be easily tested (like network access)

The `monkeypatch` feature helps to safely set/delete an attribute, dictionary item, to modify environment variables, modfiy `sys.path`, etc.

The [monkeypath blog post](http://tetamap.wordpress.com/2009/03/03/monkeypatching-in-unit-tests-done-right/) contains extra material and motivations

## Simple example: monkeypatching functions

In this case, `os.expanduser` is expected to return a certain directory

The `monkeypatch.setattr()` method can patch this function before calling into a function which uses it:

In [ ]:
# content of test_module.py
import os.path
def getssh(): # pseudo application code
    return os.path.join(os.path.expanduser("~admin"), '.ssh')

def test_mytest(monkeypatch):
    def mockreturn(path):
        return '/abc'
    monkeypatch.setattr(os.path, 'expanduser', mockreturn)
    x = getssh()
    assert x == '/abc/.ssh'

The test function monkeypatches `os.path.expanduser` and then calls into a function that calls it

After the test finishes, the `os.path.expanduser` modification will be undone

## Example: preventing  "requests" from remote operations

To prevent the "requests" library from performing http requests in all tests:

In [ ]:
# content of conftest.py
import pytest
@pytest.fixture(autouse=True)
def no_requests(monkeypatch):
    monkeypatch.delattr("requests.sessions.Session.request")

The autouse fixture will then be executed for each test function and will delete the method `request.session.Session.request` so that any attempts within a test to create http requests will fail

## Things that might break pytest

Do not patch things like `open`, `compile`, etc.

Patching `stdlib` functions and some third-party libraries used by pytest might break pytest, so it is recommended to use `MonkeyPatch.context()` to limit the patching to the necessary block:

In [ ]:
import functools


def test_partial(monkeypatch):
    with monkeypatch.context() as m:
        m.setattr(functools, "partial", 3)
        assert functools.partial == 3

## Additional references

[MonkeyPatch class](https://docs.pytest.org/en/latest/reference.html#_pytest.monkeypatch.MonkeyPatch)

[stackoverflow pytest thread](https://stackoverflow.com/search?q=pytest)